- partial 源码解析
- 用一个实际的例子，加一些打印信息来说明，注释都是基于该例子
- 在Debug 模式下阅读更加


In [3]:
class partial:
    """New function with partial application of the given arguments
    and keywords.
    """

    __slots__ = "func", "args", "keywords", "__dict__", "__weakref__"

    def __new__(*args, **keywords):
        """  加了一些注释"""
        """此时 args 包括2个参数,一个partial实例  一个add 函数"""
        if not args:
            raise TypeError("descriptor '__new__' of partial needs an argument")

        """ 这里明显是 限制必须输入一个被装饰的函数 """
        if len(args) < 2:
            raise TypeError("type 'partial' takes at least one argument")

        """ cls：实例名   func :被装饰的函数 *args 为空???? """
        cls, func, *args = args

        """ func 必须是一个可以调用的函数 """
        if not callable(func):
            raise TypeError("the first argument must be callable")

        args = tuple(args)
        print(args)

        if hasattr(func, "func"):
            print(func.args)
            args = func.args + args
            tmpkw = func.keywords.copy()
            print(tmpkw)
            tmpkw.update(keywords)
            keywords = tmpkw
            del tmpkw
            func = func.func

        self = super(partial, cls).__new__(cls)

        self.func = func
        self.args = args
        self.keywords = keywords
        return self

    def __call__(*args, **keywords): ## __call__ 实例调用时返回的方法

        """调用拿 print(add_X1_2(x2=1,x3=2)) 举例  """
        if not args:
            raise TypeError("descriptor '__call__' of partial needs an argument")

        """ self.keywords:{X1:3}       keywords:{x2=1,x3=2}          newkeywords{X1:3,x2=1,x3=2} """
        self, *args = args
        newkeywords = self.keywords.copy()
        newkeywords.update(keywords)
        """ return add((),[],{X1:3,x2=1,x3=2} )"""
        return self.func(*self.args, *args, **newkeywords)




def add(x1,x2,x3):
    return x1+x2+x3

add_X1_2 = partial(add(1),x1=3)

print(add_X1_2(x2=1,x3=2))
print(add_X1_2(x2=2,x3=3))


- fun 的一些参数


In [2]:
def func_test(param=1):
    print(param)

def func_test_(**kwargs):
    print(**kwargs)

print(func_test.__dict__)

print(func_test_.__dict__)

print(func_test.__call__)


class Test():
    def __new__(*args, **kwargs):
        print(args,kwargs)

        cls, func, *args = args
        print(cls,func,*args)


a = Test(func_test,value=1,value2="#")

{}
{}
<method-wrapper '__call__' of function object at 0x05D81C90>
(<class '__main__.Test'>, <function func_test at 0x05D81C90>) {'value': 1, 'value2': '#'}
<class '__main__.Test'> <function func_test at 0x05D81C90>
